# QuestionOrNot: English Text Classifier

### The goal of this project is to build a machine learning model that is able to predict if the given sentence is a question or not. All details can be found in the project report.

### Accuracy of the built classifier:

Accuracy:  0.7910179640718563
Error Rate:  0.20898203592814368
Precision:  0.8260572987721692
Recall:  0.7321644498186215
F1 =  0.7901073177758571

In [22]:
import pandas as pd
import nltk.data;
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import word2vec;
from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;
import pandas as pd;
import numpy as np;
import os;
import re;
import logging;
import sqlite3;
import time;
import sys;
import multiprocessing;
import matplotlib.pyplot as plt;
from itertools import cycle;
import numpy
import io
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [32]:
dataframe = pd.read_csv('CPCS433Dataset.csv')
dataframe.rename(columns = {'sentence ':'sentence'}, inplace = True)

#change to lower case
dataframe['sentence'] = dataframe['sentence'].str.lower()

In [33]:
dataframe.head()

,sentence,type
0,is 3hrill com brandable for a t shirt business...,Q
1,why effective app support is necessary for you...,Q
2,what business can be set up next to a car show...,Q
3,how should i report a vending machine competitor,Q
4,how can i retain younger employees with least ...,Q


In [34]:
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
dataframe.head(10)


,sentence,type
0,ii a midnight autumns dream is italian power m...,S
1,her disinherited nephew pony duke and jason th...,S
2,in the year 2000 it became an independent club,S
3,kaleidoscope heart is the third studio album a...,S
4,is facebooks decision to algorithmically selec...,Q
5,what are some mind blowing bike inventions,Q
6,what are some examples of positive effects of ...,Q
7,it took place between indonesias declaration o...,S
8,what can i do to improve my knowledge of thing...,Q
9,other volunteers including tejano and texian c...,S


In [35]:
# The frequency distribution of the words
sentences = dataframe.sentence.str.cat(sep=' ')

#Tokenize sentences
tokens = nltk.word_tokenize(sentences)

#Get frequency distribution
freqDist = nltk.FreqDist(tokens)

In [36]:
dataframe['tokenized_sents'] = dataframe.apply(lambda row: nltk.word_tokenize(row['sentence']), axis=1)
dataframe.head()

,sentence,type,tokenized_sents
0,ii a midnight autumns dream is italian power m...,S,"[ii, a, midnight, autumns, dream, is, italian,..."
1,her disinherited nephew pony duke and jason th...,S,"[her, disinherited, nephew, pony, duke, and, j..."
2,in the year 2000 it became an independent club,S,"[in, the, year, 2000, it, became, an, independ..."
3,kaleidoscope heart is the third studio album a...,S,"[kaleidoscope, heart, is, the, third, studio, ..."
4,is facebooks decision to algorithmically selec...,Q,"[is, facebooks, decision, to, algorithmically,..."


In [19]:
# download the model and return as object ready for use
model_glove_twitter = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [20]:
model_gigaword = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [30]:
def sentence_into_vec(df):
  # Iterate over each row
  for index, row in df.iterrows():
    # Extract cell
    sent = row['tokenized_sents']

    # Vectorize
    try:
      vectors = [model_gigaword[x] for x in sent]
    except:
        ""

    vec = []
    for v in vectors:
      vec = np.concatenate((vec, v), axis=0)

    # Save vector over cell
    row['tokenized_sents'] = vec

In [37]:
#vectorize sentences
sentence_into_vec(dataframe)

#check vectors
dataframe.head()

,sentence,type,tokenized_sents
0,ii a midnight autumns dream is italian power m...,S,"[-0.08455000072717667, 0.4552899897098541, -0...."
1,her disinherited nephew pony duke and jason th...,S,"[0.33390000462532043, -0.5213599801063538, 0.2..."
2,in the year 2000 it became an independent club,S,"[0.08570300042629242, -0.22201000154018402, 0...."
3,kaleidoscope heart is the third studio album a...,S,"[0.08570300042629242, -0.22201000154018402, 0...."
4,is facebooks decision to algorithmically selec...,Q,"[0.08570300042629242, -0.22201000154018402, 0...."


In [38]:
#Get maximum length of vectors
max_shape = 0
for index, row in dataframe.iterrows():
    sent = row['tokenized_sents']
    current = sent.shape[0]
    if current > max_shape:
      max_shape = current
print("Maximum shape: ",max_shape)

Maximum length:  3900


In [39]:
# First Encode label 
encoder = preprocessing.LabelEncoder()
dataframe['type'] = encoder.fit_transform(dataframe['type'])

features = dataframe['tokenized_sents']
label = dataframe['type']

featuresP = pad_sequences(features)

x_train, x_test, y_train, y_test = train_test_split(featuresP, label, train_size=0.8)

#Train SVM Model

from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train,y_train)
predictions_SVM = SVM.predict(x_test)


In [41]:
# Get accuracy
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

acc = accuracy_score(predictions_SVM, y_test)
print("Accuracy: ", acc)
print("Error Rate: ", 1 - acc)
print("Precision: ", precision_score(y_test, predictions_SVM))
print("Recall: ", recall_score(y_test, predictions_SVM))
print("F1 = ", f1_score(y_test, predictions_SVM , average="macro"))
print("Confusion matrix", confusion_matrix(y_test, predictions_SVM))

Accuracy:  0.7910179640718563
Error Rate:  0.20898203592814368
Precision:  0.8260572987721692
Recall:  0.7321644498186215
F1 =  0.7901073177758571
Confusion matrix [[1431  255]
 [ 443 1211]]
